In [0]:
%tensorflow_version 2.x

In [0]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [4]:
# Load Fashine MNIST
(X_train, y_train0), (X_test, y_test0) = datasets.fashion_mnist.load_data()

# Reshape size
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

# Normalize the range of featurs
X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate, add    # 為了分歧

In [0]:
x = Input(shape=(28,28,1))

cnn_1_1 = Conv2D(32, (3, 3), padding='same', activation='relu')
max_pool_1_1 = MaxPool2D()
cnn_1_2 = Conv2D(32, (3, 3), padding='same', activation='relu')
max_pool_1_2 = MaxPool2D()


cnn_2 = Conv2D(64, (3, 3), padding='same', activation='relu')
max_pool_2 = MaxPool2D()
cnn_3 = Conv2D(128, (3, 3), padding='same', activation='relu')
max_pool_3 = MaxPool2D()
gap = GlobalAveragePooling2D()
dense = Dense(10, activation='softmax')

In [0]:
# funcitonal API 方式連接

# 分歧點
f1_1 = max_pool_1_1(cnn_1_1(x))
f1_2 = max_pool_1_2(cnn_1_2(x))

# 合併
z = concatenate([f1_1, f1_2])

f2 = max_pool_2(cnn_2(z))
f3 = max_pool_3(cnn_3(f2))
f4 = gap(f3)
y = dense(f4)

In [0]:
# model 化

model_functional = Model(x, y)

In [9]:
model_functional.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 32)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [0]:
model_functional.compile(loss='mse', 
              optimizer=Adam(), 
              metrics=['accuracy'])

In [11]:
model_functional.fit(X_train, y_train, 
          batch_size=256, 
          epochs=5)

Epoch 1/5
235/235 [==============================] - 6s 24ms/step - loss: 0.0439 - accuracy: 0.6752
Epoch 2/5
235/235 [==============================] - 5s 23ms/step - loss: 0.0268 - accuracy: 0.8115
Epoch 3/5
235/235 [==============================] - 5s 23ms/step - loss: 0.0229 - accuracy: 0.8427
Epoch 4/5
235/235 [==============================] - 5s 23ms/step - loss: 0.0209 - accuracy: 0.8562
Epoch 5/5
235/235 [==============================] - 5s 23ms/step - loss: 0.0195 - accuracy: 0.8666


In [12]:
score = model_functional.evaluate(X_test, y_test, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

1/1 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 0.8641
Loss: 0.019790399819612503
準確率: 86.40999794006348
